## Notebook Purpose:

1. The primary goal of this notebook is to validate the results produced by the platform.
2. We are picking one experiment = `exp_big_exp_run` and we are going to validate following:
    - AB Results
3. This validation will ensure if the platform is trustworthy or not.



In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [ ]:

from pyspark.sql import SparkSession
spark = (
        SparkSession.builder
        .appName("Laptop_ROG_AB_validation")
        .master("spark://10.0.0.80:7077")

        .config("spark.executor.instances", "2")
        .config("spark.executor.cores", "10")
        .config("spark.executor.memory", "18g")
        .config("spark.executor.memoryOverhead", "4g")

        .config("spark.driver.memory", "10g")
        .config("spark.driver.maxResultSize", "2g")
        .config("spark.driver.host", "10.0.0.80")
        .config("spark.driver.bindAddress", "0.0.0.0")

        # AQE + shuffle
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.shuffle.partitions", "288")
        .config("spark.sql.files.maxPartitionBytes", "256m")

        # Don’t set spark.local.dir here; use SPARK_LOCAL_DIRS on the worker
        # .config("spark.local.dir", "...")  <-- remove
            # -------- MinIO / S3A (must match Iceberg) --------
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
        .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
        .config("spark.hadoop.fs.s3a.endpoint", "http://10.0.0.80:9100")  # <-- changed
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config("spark.hadoop.fs.s3a.aws.credentials.provider",
                "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

        # -------- Iceberg REST catalog --------
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.iceberg.type", "hadoop")
        # .config("spark.sql.catalog.iceberg.uri", "http://10.0.0.59:8181")
        .config("spark.sql.catalog.iceberg.warehouse", "s3a://iceberg-warehouse/warehouse/")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", "iceberg")
        .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")

        # Iceberg's own S3 settings (again, pointing to MinIO)
        .config("spark.sql.catalog.iceberg.s3.endpoint", "http://10.0.0.80:9100")
        .config("spark.sql.catalog.iceberg.s3.path-style-access", "true")
        .config("spark.sql.catalog.iceberg.s3.access-key-id", "minioadmin")
        .config("spark.sql.catalog.iceberg.s3.secret-access-key", "minioadmin")
        .config(
            "spark.jars",
            "/opt/spark/jars/iceberg-spark-runtime-3.4_2.12-1.6.0.jar,"
            "/opt/spark/jars/iceberg-aws-bundle-1.6.0.jar")

        .getOrCreate()
    )

26/02/18 23:43:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Check that tables exists and we are picking experiment registry to check experiment data exists:
spark.sql("Show tables in iceberg.exp").show(truncate=False)

+---------+-------------------------+-----------+
|namespace|tableName                |isTemporary|
+---------+-------------------------+-----------+
|exp      |analysis_results         |false      |
|exp      |did_results              |false      |
|exp      |experiment_params        |false      |
|exp      |outcomes                 |false      |
|exp      |analysis_results_cuped   |false      |
|exp      |experiment_registry      |false      |
|exp      |exposures                |false      |
|exp      |metric_aggregates_daily  |false      |
|exp      |metric_aggregates_overall|false      |
+---------+-------------------------+-----------+



In [ ]:
spark.sql("Select * from iceberg.exp.experiment_registry where experiment_id  = 'exp_big_exp_run'").show(truncate=False)

# So the experiment does exist in the table:

+---------------+--------------------------+-------------------+-------------------+-----+--------------------+---------------+-----------------+-------------------------+
|experiment_id  |name                      |start_ts           |end_ts             |owner|variants            |primary_metrics|guardrail_metrics|notes                    |
+---------------+--------------------------+-------------------+-------------------+-----+--------------------+---------------+-----------------+-------------------------+
|exp_big_exp_run|Checkout button color test|2026-01-01 00:00:00|2026-01-29 00:00:00|you  |[control, treatment]|[conversion]   |[revenue]        |Synthetic demo experiment|
+---------------+--------------------------+-------------------+-------------------+-----+--------------------+---------------+-----------------+-------------------------+



In [9]:
# Cursory view of exposures:

spark.sql("select * from iceberg.exp.exposures limit 10").show(truncate=False)

+----------------------+---------+---------+-------------------+-------+
|experiment_id         |user_id  |variant  |exposure_ts        |unit   |
+----------------------+---------+---------+-------------------+-------+
|exp_precision_long_run|u_0000000|treatment|2026-01-07 03:53:33|user_id|
|exp_precision_long_run|u_0000002|control  |2026-01-20 19:53:12|user_id|
|exp_precision_long_run|u_0000003|control  |2026-01-18 12:01:39|user_id|
|exp_precision_long_run|u_0000004|treatment|2026-01-16 20:53:58|user_id|
|exp_precision_long_run|u_0000006|control  |2026-01-18 18:29:13|user_id|
|exp_precision_long_run|u_0000007|treatment|2026-01-16 17:38:18|user_id|
|exp_precision_long_run|u_0000008|treatment|2026-01-03 20:18:39|user_id|
|exp_precision_long_run|u_0000009|treatment|2026-01-06 22:58:37|user_id|
|exp_precision_long_run|u_0000010|treatment|2026-01-19 12:22:58|user_id|
|exp_precision_long_run|u_0000011|control  |2026-01-24 04:13:55|user_id|
+----------------------+---------+---------+-------

In [ ]:
#  Lets validate the number of users in treatment vs control:

spark.sql("select variant, count(distinct user_id) from iceberg.exp.exposures group by variant").show(truncate=False)

#  Looks like there: 482063 User in treatment and 480563 users in control

+---------+-----------------------+
|variant  |count(DISTINCT user_id)|
+---------+-----------------------+
|control  |480563                 |
|treatment|482063                 |
+---------+-----------------------+



In [ ]:
# Lets check the outcomes table that checks results of exposures:

spark.sql("select * from iceberg.exp.outcomes limit 10").show(truncate=False)

#  For our validation we will focus on two metrics: conversions (primary NSM) and revenue (guardrail)

+--------------------+---------+-----------+------------------+-------------------+
|experiment_id       |user_id  |metric_name|value             |ts                 |
+--------------------+---------+-----------+------------------+-------------------+
|exp_pricing_test_003|u_0000000|pre_revenue|19.06501624999786 |2025-12-20 02:52:10|
|exp_pricing_test_003|u_0000001|pre_revenue|11.238445754479702|2025-12-25 13:53:17|
|exp_pricing_test_003|u_0000002|pre_revenue|22.18493197243855 |2025-12-24 12:22:39|
|exp_pricing_test_003|u_0000003|pre_revenue|18.2811468399141  |2025-12-27 18:20:52|
|exp_pricing_test_003|u_0000004|pre_revenue|10.860730895283062|2025-12-30 01:01:11|
|exp_pricing_test_003|u_0000005|pre_revenue|17.73578908053096 |2025-12-22 03:05:08|
|exp_pricing_test_003|u_0000006|pre_revenue|14.43048618778002 |2025-12-28 20:09:49|
|exp_pricing_test_003|u_0000007|pre_revenue|6.300105356920774 |2025-12-27 18:11:02|
|exp_pricing_test_003|u_0000008|pre_revenue|6.9575732084637725|2025-12-23 07

In [ ]:
# Lets get a quick count:

spark.sql("select metric_name, count(distinct user_id) from iceberg.exp.outcomes group by metric_name").show(truncate=False)

#  We are ignoring pre-revenue for now. but it becomes important for CUPED and DiD analysis
# Looks like conversion and revenue has 875355 which is expected since we track revenue only for converted users.

+-----------+-----------------------+
|metric_name|count(DISTINCT user_id)|
+-----------+-----------------------+
|conversion |875355                 |
|pre_revenue|1000000                |
|revenue    |875355                 |
+-----------+-----------------------+



So far we have validated that there are exposures and outcomes and the sample sizes. Now we we dive into AB testing validations.

## AB Testing Validation:

1. We will validate the summary produced by our platform i.e we get same mean, delta and relative lift by querying tables independently.
2. Validate the results from statistical test using independent tooling such as Scipy and Numpy.

### Summary Table
` Here are the results produced by the platform:`

| metric | control mean | treatment mean | delta | rel lift | CI low | CI high | p-value | method |
|---|---:|---:|---:|---:|---:|---:|---:|---|
| conversion | 0.0999198 | 0.150077 | 0.0501574 | 0.501976 | 0.0487552 | 0.0515595 | 0 | two_proportion_z |
| pre_revenue | 12.4993 | 12.5042 | 0.00491569 | 0.000393278 | -0.00782836 | 0.0176597 | 0.449637 | welch_normal_approx |
| revenue | 1.25252 | 1.87712 | 0.624599 | 0.498674 | 0.606476 | 0.642723 | 0 | welch_normal_approx |

In [13]:
#  Import Libraries
import scipy
import numpy as np

In [19]:
# We need to JOIN exposures and outcomes to measure the results

v_sql = """ create or replace temporary view ab_results as

select exposures.experiment_id, exposures.user_id, outcomes.metric_name, outcomes.value, outcomes.ts, exposures.variant

from iceberg.exp.exposures as exposures
inner join iceberg.exp.outcomes as outcomes
on exposures.experiment_id = outcomes.experiment_id
and exposures.user_id = outcomes.user_id
and exposures.experiment_id = 'exp_big_exp_run'
and outcomes.metric_name in ('revenue','conversion')

"""

spark.sql(v_sql).show(truncate=False)
spark.sql("select * from ab_results limit 10").show(truncate=False)

#  So the query returns data. But we need to check if there are any user_id, metric_name duplicates. Refer to next cell for details:

++
||
++
++



26/02/19 00:40:26 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+---------------+---------+-----------+-----+-------------------+---------+
|experiment_id  |user_id  |metric_name|value|ts                 |variant  |
+---------------+---------+-----------+-----+-------------------+---------+
|exp_big_exp_run|u_0000001|conversion |0.0  |2026-01-13 14:34:21|treatment|
|exp_big_exp_run|u_0000001|revenue    |0.0  |2026-01-13 14:34:21|treatment|
|exp_big_exp_run|u_0000002|conversion |0.0  |2026-01-24 23:59:10|treatment|
|exp_big_exp_run|u_0000002|revenue    |0.0  |2026-01-24 23:59:10|treatment|
|exp_big_exp_run|u_0000003|conversion |0.0  |2026-01-14 08:00:47|control  |
|exp_big_exp_run|u_0000003|revenue    |0.0  |2026-01-14 08:00:47|control  |
|exp_big_exp_run|u_0000004|conversion |0.0  |2026-01-04 21:18:41|treatment|
|exp_big_exp_run|u_0000004|revenue    |0.0  |2026-01-04 21:18:41|treatment|
|exp_big_exp_run|u_0000005|conversion |0.0  |2026-01-20 23:02:50|control  |
|exp_big_exp_run|u_0000005|revenue    |0.0  |2026-01-20 23:02:50|control  |
+-----------

In [20]:
#  Please note that following SQL return blank:
spark.sql("select user_id, metric_name, count(*) from ab_results group by user_id, metric_name having count(*)> 1").show(truncate=False)

26/02/19 00:40:33 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+-------+-----------+--------+
|user_id|metric_name|count(1)|
+-------+-----------+--------+
+-------+-----------+--------+



### Validation Flow:
 For validating AB test results we need:

#### Conversions: 
we will be using Two Proportions Z-Test as the metric is BINARY:
1. We will be using statsmodels for this test: `from statsmodels.stats.proportion import proportions_ztest`
2. Calculate total users, conversions per variant. This will give effect of the treatment, relative lift.
3. `proportions_ztest` will give us the z and p values.



In [29]:
v_sql = """ 
select variant, metric_name, count(*) as total_users,
count(distinct(case when value <> 0 then user_id else null end)) as converted_users,
count(distinct(case when value <> 0 then user_id else null end))/count(*)  as conversion_proportion
from ab_results where metric_name = 'conversion'
group by variant, metric_name
"""



spark.sql(v_sql).show(truncate=False)

ab_summary = spark.sql(v_sql).toPandas()

ab_summary.sort_values(by = 'variant', inplace=True, ascending=False)

#  Please note that

26/02/19 01:00:01 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+---------+-----------+-----------+---------------+---------------------+
|variant  |metric_name|total_users|converted_users|conversion_proportion|
+---------+-----------+-----------+---------------+---------------------+
|control  |conversion |424190     |42385          |0.09991984723826587  |
|treatment|conversion |425521     |63861          |0.15007719948016668  |
+---------+-----------+-----------+---------------+---------------------+



26/02/19 01:00:02 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


In [30]:
ab_summary

,variant,metric_name,total_users,converted_users,conversion_proportion
1,treatment,conversion,425521,63861,0.150077
0,control,conversion,424190,42385,0.099920


In [47]:
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import confint_proportions_2indep
import pandas as pd

successes = ab_summary['converted_users']
n_obs = ab_summary['total_users']
print(f"Success Series: {successes}")
print(f"N Obs Series: {n_obs}")

z_stat, p_value = proportions_ztest(count=successes, nobs=n_obs, alternative='two-sided')
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

lower_bound, upper_bound = confint_proportions_2indep(63861, 425521, 42385, 424190, alpha=0.05, compare='diff')

print(f"CI Lower Bound:  {lower_bound}")
print(f"CI Upper Bound:  {upper_bound}")

ab_analysis_dict = {}
ab_analysis_dict['Control Mean'] = [ab_summary[ab_summary['variant']=='control']['conversion_proportion'].values[0]]
ab_analysis_dict['Treatment Mean'] = [ab_summary[ab_summary['variant']=='treatment']['conversion_proportion'].values[0]]
ab_analysis_dict['Delta'] = [(ab_summary[ab_summary['variant']=='treatment']['conversion_proportion'].values[0] 
                             - ab_summary[ab_summary['variant']=='control']['conversion_proportion'].values[0])]

ab_analysis_dict['Relative Lift']  = [ab_analysis_dict['Delta']/ab_summary[ab_summary['variant']=='control']['conversion_proportion'].values[0]]
ab_analysis_dict['CI Lower Bound']  = [lower_bound]
ab_analysis_dict['CI Upper Bound']  = [upper_bound]
ab_analysis_dict['p-Value'] = [p_value]
ab_analysis_dict['z-stat'] = [z_stat]
ab_analysis_dict

ab_analysis = pd.DataFrame(ab_analysis_dict)
# ab_analysis_dict
ab_analysis



Success Series: 1    63861
0    42385
Name: converted_users, dtype: int64
N Obs Series: 1    425521
0    424190
Name: total_users, dtype: int64
Z-statistic: 69.8915
P-value: 0.0000
CI Lower Bound:  0.048755296352205955
CI Upper Bound:  0.05155957941286664


,Control Mean,Treatment Mean,Delta,Relative Lift,CI Lower Bound,CI Upper Bound,p-Value,z-stat
0,0.09992,0.150077,0.050157,[0.5019758699420055],0.048755,0.05156,0.0,69.891502


#### Compare above results with platform results:
| metric | control mean | treatment mean | delta | rel lift | CI low | CI high | p-value | method |
|---|---:|---:|---:|---:|---:|---:|---:|---|
| conversion | 0.0999198 | 0.150077 | 0.0501574 | 0.501976 | 0.0487552 | 0.0515595 | 0 | two_proportion_z |

Based on comparison we can say that conversion is a pass.

#### Revenue Measurement:

1. Since this is a continuous variable we will be using Welch's (unequal variances) test for its robustness.
2. For this we will using SCIPY



In [51]:
v_sql = """ 
select variant, metric_name, 
avg(value) as mean_value
from ab_results where metric_name = 'revenue'
group by variant, metric_name
"""



spark.sql(v_sql).show(truncate=False)

ab_summary = spark.sql(v_sql).toPandas()

ab_summary.sort_values(by = 'variant', inplace=True, ascending=False)

26/02/19 01:20:40 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+---------+-----------+------------------+
|variant  |metric_name|mean_value        |
+---------+-----------+------------------+
|treatment|revenue    |1.8771188733014121|
|control  |revenue    |1.2525195406211276|
+---------+-----------+------------------+



26/02/19 01:20:40 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


In [54]:
v_sql = """
select variant, value
from ab_results
where metric_name = 'revenue'

"""
# spark.sql(v_sql).show(truncate=False)

reve_data = spark.sql(v_sql).toPandas()
reve_data


26/02/19 01:22:18 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


,variant,value
0,treatment,0.0
1,treatment,0.0
2,control,0.0
3,treatment,0.0
4,control,0.0
...,...,...
849706,treatment,0.0
849707,control,0.0
849708,treatment,0.0
849709,control,0.0


In [59]:
reve_data[reve_data['variant']=='control']['value']

2         0.0
4         0.0
5         0.0
8         0.0
11        0.0
         ... 
849703    0.0
849705    0.0
849707    0.0
849709    0.0
849710    0.0
Name: value, Length: 424190, dtype: float64

In [66]:
control_values

array([0., 0., 0., ..., 0., 0., 0.])

In [71]:
from scipy import stats

control_values = np.array(reve_data[reve_data['variant']=='control']['value'])
treatment_values = np.array(reve_data[reve_data['variant']=='treatment']['value'])


r  = stats.ttest_ind(treatment_values, control_values, equal_var=False)
ci = r.confidence_interval(confidence_level=0.95)

print(f"Control Mean: {np.mean(control_values)}")
print(f"Treatment Mean: {np.mean(treatment_values)}")
print(f"Delta:   {np.mean(treatment_values) - np.mean(control_values)}")
print(f"Relative Lift:  {(np.mean(treatment_values) - np.mean(control_values))/np.mean(control_values)}")
print(f"Calculated T-statistic: {r.statistic:.3f}")
print(f"P-value: {r.pvalue:.3f}")
print(f"95% Confidence Interval for the difference in means: ({ci.low:.6f}, {ci.high:.6f})")

Control Mean: 1.2525195406211218
Treatment Mean: 1.8771188733013897
Delta:   0.6245993326802679
Relative Lift:  0.49867432197547223
Calculated T-statistic: 67.548
P-value: 0.000
95% Confidence Interval for the difference in means: (0.606476, 0.642723)


#### Compare Above results with Platform results:

| metric | control mean | treatment mean | delta | rel lift | CI low | CI high | p-value | method |
|---|---:|---:|---:|---:|---:|---:|---:|---|
| revenue | 1.25252 | 1.87712 | 0.624599 | 0.498674 | 0.606476 | 0.642723 | 0 | welch_normal_approx |